In [1]:
using ReadableRegex
using Plots
currentpath = pwd()

"/home/davibarreira/MEGA/EMAp/PlutoLatexConverter.jl"

In [2]:
module Runner
end

Main.Runner

In [3]:
figureindex = 0

0

In [30]:
function extractnotebook(notebook, notebookname=nothing)
    s = read(notebook, String)
    cells = split(s, "# ╔═╡ ");
    # The first cell and the final 3 are not used
    codes, contents, outputtag, celltype = [],[],[],[]
    for cell in cells[2:end-3]
        push!(codes, cell[1:36])
        push!(contents, cell[38:end])
        push!(outputtag, endswith(rstrip(cell),";") ? "hideoutput" : "showoutput")
        push!(celltype, cell[38:42] == "md\"\"\"" ? "markdown" : "code")
    end
    
    # Get order and view type
    r = either("# ╠","# ╟")
    sortedcells = split(cells[end],Regex(r))
    sortedcodes = [cell[4:39] for cell in sortedcells[2:end-2]]
    order = [findfirst(isequal(scode),codes) for scode in  sortedcodes[1:end]]
    view  = [occursin("═",c) ? "showcode" : "hidecode" for c in sortedcells[2:end-2]]
    # Matching running order
    view  = view[order]
    
    if notebookname === nothing
        notebookname = split(notebook,"/")[end]
        if endswith(notebookname,".jl")
            notebookname = notebookname[1:end-3]
        end
    end
    
    notebookdata = Dict(:codes => codes, :notebookname => notebookname,
                        :contents => contents, :outputtag=>outputtag,
                        :celltype => celltype,:order=> order,:view=>view)
    return notebookdata
end

extractnotebook (generic function with 2 methods)

In [31]:
function collectoutputs(nb, notebookfolder="./")
    runpath = pwd()
    cd(notebookfolder)
    outputs = []
    for (i, content) ∈ enumerate(nb[:contents])
        if nb[:celltype][i] == "code"
            if startswith(lstrip(content),"begin") && endswith(rstrip(content),"end")
                ex = :($(Meta.parse(strip(content))))
            else
                ex = :($(Meta.parse("begin\n"*content*"\nend")))
            end
            
            s = string(ex.args[end])
            if contains(s, Regex(either("PlutoUI.LocalResource","LocalResource")))
                imagepath = s[findfirst(Regex(look_for(one_or_more(ANY),after="(",before=")")),s)]
                push!(outputs,("imagepath",imagepath))
            else
                println(nb[:contents][i])
                io = IOBuffer();
                Base.invokelatest(show,IOContext(io, :limit => true),
                    "text/plain", dispatch_output(Runner.eval(ex), nb[:notebookname]));
                celloutput = String(take!(io))
                if celloutput == "nothing"
                    celloutput = ""
                elseif startswith(celloutput, "Plot{Plots.")
                    celloutput = ""
                end
                push!(outputs, celloutput)
            end
        end
    end
    cd(runpath)
    return outputs
end

function dispatch_output(command_eval::Plots.Plot, notebookname)
    global figureindex+=1
    savefig(command_eval,notebookname*"_"*"figure"*string(figureindex)*".png")
    return command_eval
end

function dispatch_output(command_eval, notebookname)
   return command_eval 
end

dispatch_output (generic function with 4 methods)

In [32]:
example = "./example/examplepluto.jl"
nb = extractnotebook(example)

Dict{Symbol, Any} with 7 entries:
  :codes        => Any["7b4b693d-c955-4c9b-97e8-56d28cf74e18", "bdad72d3-5744-4…
  :outputtag    => Any["showoutput", "showoutput", "showoutput", "showoutput", …
  :celltype     => Any["code", "code", "markdown", "code", "code", "code", "cod…
  :view         => ["showcode", "showcode", "showcode", "showcode", "showcode",…
  :order        => [3, 1, 2, 4, 5, 6, 7, 8]
  :notebookname => "examplepluto"
  :contents     => Any["using PlutoUI\n\n", "\n\t begin\n\tusing Plots\n\ty(x) …

In [33]:
cd(currentpath)
outputs = collectoutputs(nb,"./example/");

using PlutoUI



	 begin
	using Plots
	y(x) = sin(x)
	plot(y,
		color=:blue)
end



A = [10,10,10]


x = rand(10);


x .+ 1




In [29]:
outputs

7-element Vector{Any}:
 ""
 ""
 "3-element Vector{Int64}:\n 10\n 10\n 10"
 "10-element Vector{Float64}:\n 0.011484090200442054\n 0.01820331571644096\n 0.5318877331618481\n 0.11651410662386796\n 0.1890888653818048\n 0.5285985305739234\n 0.37688298409874244\n 0.07979615914345284\n 0.5463733032020945\n 0.5411381912886879"
 "10-element Vector{Float64}:\n 1.011484090200442\n 1.018203315716441\n 1.5318877331618481\n 1.116514106623868\n 1.1890888653818048\n 1.5285985305739234\n 1.3768829840987424\n 1.0797961591434528\n 1.5463733032020945\n 1.541138191288688"
 ("imagepath", "\"figure.svg\"")
 ("imagepath", "\"plotexample.png\"")